#Модуль 17

In [212]:
#!pip install requests

##Создаю пандас датафреймы с данными

In [213]:
import requests
import json
import pandas as pd

In [214]:
req = requests.get('https://openexchangerates.org/api/latest.json?app_id=44b978a695b149198655f1beb73a770c')

data1 = json.loads(req.text)
df_rates = pd.read_json(json.dumps(data1))
df_rates.reset_index(inplace=True)
df_rates.drop(columns=['disclaimer', 'license', 'timestamp', 'base'], inplace=True)
df_rates.rename(columns={'index':'currency'}, inplace=True)
df_rates.head()

,currency,rates
0,AED,3.673000
1,AFN,70.905322
2,ALL,92.375041
3,AMD,387.284147
4,ANG,1.797650


In [215]:
df_clients = pd.read_excel('CASE_CLIENTS.xlsx')
df_clients.head()

,client_id,last_name,first_name,middle_name,age,sex
0,10000041307,З*****,Р*****,О*****,30,F
1,10000133158,И*****,М*****,Ю*****,37,M
2,100003773,Е*****,О*****,С*****,50,M
3,10000535718,Е*****,Ф*****,Ч*****,45,M
4,10000583119,И*****,Ф*****,Ф*****,28,M


In [216]:
df_contracts = pd.read_excel('CASE_CONTRACTS.xlsx')
df_contracts.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000


In [217]:
df_losses = pd.read_excel('CASE_LOSSES.xlsx')
df_losses.head()

,loss_id,client_id,loss_name,loss_payout_amt
0,1,10000713193,Оказание медицинской помощи,100000
1,2,100014656,Оказание медицинской помощи,50000
2,3,10006545463,Оказание медицинской помощи,30000
3,4,10006611194,Оказание медицинской помощи,70000
4,5,1001091597,Оказание медицинской помощи,60000


##Приступаю к созданию витрины данных

###Объединяю датафреймы

In [218]:
df = df_contracts.merge(df_clients, on='client_id')
df = df.merge(df_losses, on='client_id', how='left')
df.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,last_name,first_name,middle_name,age,sex,loss_id,loss_name,loss_payout_amt
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000,А*****,Х*****,А*****,20,M,NaN,NaN,NaN
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000,З*****,Х*****,М*****,55,M,NaN,NaN,NaN
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000,Б*****,В*****,Щ*****,61,F,NaN,NaN,NaN
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000,И*****,У*****,Н*****,25,M,NaN,NaN,NaN
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000,З*****,Ч*****,Г*****,33,M,NaN,NaN,NaN


###Провожу операции с курсами

In [219]:
#Создаю переменную с курсом рубля к доллару
exch_rate = df_rates.loc[df_rates['currency'] == 'RUB', 'rates']
exch_rate = exch_rate.values[0]

In [220]:
#Там, где значения цены и тп выражены в долларах, привожу их к рублям
df.loc[df['currency_name'] == 'Доллар США', 'price'] *= exch_rate
df.loc[df['currency_name'] == 'Доллар США', 'insurance_amount'] *= exch_rate
df.loc[df['currency_name'] == 'Доллар США', 'loss_payout_amt'] *= exch_rate

df['price'] = round(df['price'])
df['insurance_amount'] = round(df['insurance_amount'])
df['loss_payout_amt'] = round(df['loss_payout_amt'])

In [222]:
#Выкидываю столбец currency_name так как у нас теперь все в рублях и этот столбец не нужен
df.drop(columns=['currency_name'], inplace=True)
#Переименовываю столбцы с ценой и тп, чтобы обозначить, что они выражены в рублях
df.rename(columns={'price':'price_rub', 'insurance_amount':'insurance_amount_rub', 'loss_payout_amt':'loss_payout_amt_rub'}, inplace=True)

In [223]:
df.loc[df['product_name'] == 'Страхование путешественников USD', 'product_name'] = 'Страхование путешественников'

In [225]:
#Создаю столбцы с ценой и тп, но уже выраженные в долларах
df['price_usd'] = df['price_rub'] / exch_rate
df['insurance_amount_usd'] = df['insurance_amount_rub'] / exch_rate
df['loss_payout_amt_usd'] = df['loss_payout_amt_rub'] / exch_rate

###Выкидываю недействующие контракты

In [226]:
df = df[['contract_id', 'contract_num', 'product_name', 'client_id', \
         'contract_status', 'last_name', 'first_name', 'middle_name', \
         'age', 'sex', 'duration', 'country', 'price_rub', 'price_usd', \
         'insurance_amount_rub', 'insurance_amount_usd', 'loss_id', \
         'loss_name', 'loss_payout_amt_rub', 'loss_payout_amt_usd']]

df = df[df['contract_status'] == 'Действует']
df.drop(columns=['contract_status'], inplace=True)

df.head()

,contract_id,contract_num,product_name,client_id,last_name,first_name,middle_name,age,sex,duration,country,price_rub,price_usd,insurance_amount_rub,insurance_amount_usd,loss_id,loss_name,loss_payout_amt_rub,loss_payout_amt_usd
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,А*****,Х*****,А*****,20,M,10,Беларусь,1096.0,12.468111,2000000.0,22752.027709,NaN,NaN,NaN,NaN
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,З*****,Х*****,М*****,55,M,7,Индонезия,1918.0,21.819195,5000000.0,56880.069273,NaN,NaN,NaN,NaN
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Б*****,В*****,Щ*****,61,F,10,Беларусь,1096.0,12.468111,2000000.0,22752.027709,NaN,NaN,NaN,NaN
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,И*****,У*****,Н*****,25,M,14,Грузия,1534.0,17.450805,2000000.0,22752.027709,NaN,NaN,NaN,NaN
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,З*****,Ч*****,Г*****,33,M,7,Аргентина,1918.0,21.819195,5000000.0,56880.069273,NaN,NaN,NaN,NaN


##Выгружаю все в excel файл

In [227]:
df.to_excel('data.xlsx', index=False)